# Testing implementation in Github

https://github.com/gordicaleksa/pytorch-neural-style-transfer

In [2]:
! pip install  opencv-python

In [3]:
!  apt-get update && apt-get install libgl1 | Y

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]    
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Fetched 261 kB in 0s (556 kB/s)                                
Reading package lists... Done
/bin/sh: 1: Y: not found


In [2]:
import utils as utils
#from utils.video_utils import create_video_from_intermediate_results

import torch
from torch.optim import Adam, LBFGS
from torch.autograd import Variable
import numpy as np
import os
import argparse


def build_loss(neural_net, optimizing_img, target_representations, content_feature_maps_index, style_feature_maps_indices, config):
    target_content_representation = target_representations[0]
    target_style_representation = target_representations[1]

    current_set_of_feature_maps = neural_net(optimizing_img)

    current_content_representation = current_set_of_feature_maps[content_feature_maps_index].squeeze(axis=0)
    content_loss = torch.nn.MSELoss(reduction='mean')(target_content_representation, current_content_representation)

    style_loss = 0.0
    current_style_representation = [utils.gram_matrix(x) for cnt, x in enumerate(current_set_of_feature_maps) if cnt in style_feature_maps_indices]
    for gram_gt, gram_hat in zip(target_style_representation, current_style_representation):
        style_loss += torch.nn.MSELoss(reduction='sum')(gram_gt[0], gram_hat[0])
    style_loss /= len(target_style_representation)

    tv_loss = utils.total_variation(optimizing_img)

    total_loss = config['content_weight'] * content_loss + config['style_weight'] * style_loss + config['tv_weight'] * tv_loss

    return total_loss, content_loss, style_loss, tv_loss


def make_tuning_step(neural_net, optimizer, target_representations, content_feature_maps_index, style_feature_maps_indices, config):
    # Builds function that performs a step in the tuning loop
    def tuning_step(optimizing_img):
        total_loss, content_loss, style_loss, tv_loss = build_loss(neural_net, optimizing_img, target_representations, content_feature_maps_index, style_feature_maps_indices, config)
        # Computes gradients
        total_loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        return total_loss, content_loss, style_loss, tv_loss

    # Returns the function that will be called inside the tuning loop
    return tuning_step


def neural_style_transfer(config):
    
    content_path = 'sphynx_cat.jpg'
    style_path = 'adele_block_from_klimt.jpg'
    content_img_path = content_path#os.path.join(config['content_images_dir'], config['content_img_name'])
    style_img_path = style_path#os.path.join(config['style_images_dir'], config['style_img_name'])

    #out_dir_name = 'combined_' + os.path.split(content_img_path)[1].split('.')[0] + '_' + os.path.split(style_img_path)[1].split('.')[0]
    #dump_path = os.path.join(config['output_img_dir'], out_dir_name)
    #os.makedirs(dump_path, exist_ok=True)
    
    
    dump_path=''

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    content_img = utils.prepare_img(content_img_path, config['height'], device)
    style_img = utils.prepare_img(style_img_path, config['height'], device)

    if config['init_method'] == 'random':
        # white_noise_img = np.random.uniform(-90., 90., content_img.shape).astype(np.float32)
        gaussian_noise_img = np.random.normal(loc=0, scale=90., size=content_img.shape).astype(np.float32)
        init_img = torch.from_numpy(gaussian_noise_img).float().to(device)
    elif config['init_method'] == 'content':
        init_img = content_img
    else:
        # init image has same dimension as content image - this is a hard constraint
        # feature maps need to be of same size for content image and init image
        style_img_resized = utils.prepare_img(style_img_path, np.asarray(content_img.shape[2:]), device)
        init_img = style_img_resized

    # we are tuning optimizing_img's pixels! (that's why requires_grad=True)
    optimizing_img = Variable(init_img, requires_grad=True)

    neural_net, content_feature_maps_index_name, style_feature_maps_indices_names = utils.prepare_model(config['model'], device)
    print(f'Using {config["model"]} in the optimization procedure.')

    content_img_set_of_feature_maps = neural_net(content_img)
    style_img_set_of_feature_maps = neural_net(style_img)

    target_content_representation = content_img_set_of_feature_maps[content_feature_maps_index_name[0]].squeeze(axis=0)
    target_style_representation = [utils.gram_matrix(x) for cnt, x in enumerate(style_img_set_of_feature_maps) if cnt in style_feature_maps_indices_names[0]]
    target_representations = [target_content_representation, target_style_representation]

    # Start of optimization procedure
    if config['optimizer'] == 'adam':
        optimizer = Adam((optimizing_img,), lr=1e1)
        tuning_step = make_tuning_step(neural_net, optimizer, target_representations, content_feature_maps_index_name[0], style_feature_maps_indices_names[0], config)
        for cnt in range(config['num_of_iterations']):
            total_loss, content_loss, style_loss, tv_loss = tuning_step(optimizing_img)
            with torch.no_grad():
                print(f'Adam | iteration: {cnt:03}, total loss={total_loss.item():12.4f}, content_loss={config["content_weight"] * content_loss.item():12.4f}, style loss={config["style_weight"] * style_loss.item():12.4f}, tv loss={config["tv_weight"] * tv_loss.item():12.4f}')
                utils.save_and_maybe_display(optimizing_img, dump_path, config, cnt, config['num_of_iterations'], should_display=False)
    elif config['optimizer'] == 'lbfgs':
        # line_search_fn does not seem to have significant impact on result
        optimizer = LBFGS((optimizing_img,), max_iter=config['num_of_iterations'], line_search_fn='strong_wolfe')
        cnt = 0

        def closure():
            nonlocal cnt
            if torch.is_grad_enabled():
                optimizer.zero_grad()
            total_loss, content_loss, style_loss, tv_loss = build_loss(neural_net, optimizing_img, target_representations, content_feature_maps_index_name[0], style_feature_maps_indices_names[0], config)
            if total_loss.requires_grad:
                total_loss.backward()
            with torch.no_grad():
                print(f'L-BFGS | iteration: {cnt:03}, total loss={total_loss.item():12.4f}, content_loss={config["content_weight"] * content_loss.item():12.4f}, style loss={config["style_weight"] * style_loss.item():12.4f}, tv loss={config["tv_weight"] * tv_loss.item():12.4f}')
                utils.save_and_maybe_display(optimizing_img, dump_path, config, cnt, config['num_of_iterations'], should_display=False)

            cnt += 1
            return total_loss

        optimizer.step(closure)

    return dump_path

In [3]:
config = {
'optimizer': 'adam',
'init_method' : 'content',
    'height': 512,
    'model': 'vgg19',
    'content_weight': 1e5,
    'style_weight': 3e4,
    'tv_weight': 1e1,
    'saving_freq':100,
    'img_format': (4, '.jpg'),
    'content_img_name':'cat',
    'style_img_name':'klimt',
    'num_of_iterations': 500,
    
}

neural_style_transfer(config)

[2023-03-21 05:28:35.896 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:38 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-03-21 05:28:36.154 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:38 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Using vgg19 in the optimization procedure.
Adam | iteration: 000, total loss=6191529328640.0000, content_loss=      0.0000, style loss=6191457600000.0000, tv loss=71662750.0000
saving to 0000.jpg
Adam | iteration: 001, total loss=4067653582848.0000, content_loss=228682446.2891, style loss=4067236800000.0000, tv loss=188263840.0000
Adam | iteration: 002, total loss=2817524498432.0000, content_loss=600306494.1406, style loss=2816632800000.0000, tv loss=291531420.0000
Adam | iteration: 003, total loss=2064376004608.0000, content_loss=1170479882.8125, style loss=2062841520000.0000, tv loss=364028560.0000
Adam | iteration: 004, total loss=142708794

''

In [6]:
!ls -a

.
..
.git
.ipynb_checkpoints
0000.jpg
0009.jpg
01-Neural-style-transfer-tensorflow-Copy1.ipynb
01-Neural-style-transfer-tensorflow.ipynb
02-Neural-style-transfer-pytorch-Copy1.ipynb
02-Neural-style-transfer-pytorch.ipynb
Untitled.ipynb
Vassily_Kandinsky.jpeg
YellowLabradorLooking_new.jpeg
__pycache__
adele_block_from_klimt.jpg
capstone-udacity.ipynb
cat_klimt_o_adam_i_content_h_255_m_vgg19_cw_100000.0_sw_30000.0_tv_1.0.jpg
cat_klimt_o_adam_i_content_h_525_m_vgg19_cw_100000.0_sw_30000.0_tv_1.0.jpg
models.py
sphynx_cat.jpg
tensorflow-style-transfer.ipynb
utils.py
